In [ ]:
# Set up existing models
!git clone https://github.com/sithu31296/semantic-segmentation
%cd semantic-segmentation
%pip install -e .
%pip install -U gdown

In [ ]:
# Custom imports: Dataset, Preprocessor, Dataloader, Training and Loss functions

import sys
import torch
import numpy as np
from torch.utils.data import DataLoader

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:

    from git import Repo

    # Initialize the Git repository object
    repo = Repo(".", search_parent_directories=True)

    # Get the root directory of the Git project
    root_dir = repo.git.rev_parse("--show-toplevel")

    from pathlib import Path

    # Set up path for custom importer modules
    # Data Loader
    importer_module = root_dir + '/dataloader/'
    sys.path.insert(0, importer_module)
    from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

    # Loss
    loss_module = root_dir + '/trainer/loss/'
    sys.path.insert(0, loss_module)
    import loss

    # Trainer
    trainer_module = root_dir + '/trainer/'
    sys.path.insert(0, trainer_module)
    from trainer import Ai4MarsTrainer

    # Insert here your local path to the dataset (temporary)
    data_path = '/home/leeoos/Desktop/'

else: # IN_COLAB

    from google.colab import drive
    drive.mount('/content/drive')

    # On Colab the path to the module ti fixed once you have
    # corretly set up the project with gitsetup.ipynb

    # Import Loader
    fixed_path_loader = '/content/drive/MyDrive/Github/visiope/dataloader/'
    sys.path.insert(0, fixed_path_loader)
    from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

    # Import Trainer
    fixed_path_trainer = '/content/drive/MyDrive/Github/visiope/trainer/'
    sys.path.insert(0,fixed_path_trainer )
    from trainer import Ai4MarsTrainer

    # Import Loss
    fixed_path_loss = '/content/drive/MyDrive/Github/visiope/trainer/loss/'
    sys.path.insert(0, fixed_path_loss)
    import loss

    # Get Dataset as np array
    np_dataset = True

    # Get Dataset from original sources (slower)
    original_dataset = False

    if original_dataset and not np_dataset:
        # Insert here the path to the dataset on your drive
        data_path = input("Path to Dataset: ") #'/content/drive/MyDrive/foo/'

    else:
        import gdown

        # get url of np dataset (temporarerly my drive)
        url_X = 'https://drive.google.com/uc?id=1HLAQgjZbGa3lMzdyIvgykCiUJ6P4OaEX'
        url_y = 'https://drive.google.com/uc?id=1Ue74LEe0WlEnFxRcIl19WyvXR1EyYIsS'

        # download np dataset on runtime env
        data_path = '/content/dataset/'
        gdown.download(url_X, data_path, quiet=False)
        gdown.download(url_y, data_path, quiet=False)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
# Build up Ai4MarsDataset

if not IN_COLAB:

    data_import = Ai4MarsImporter()

    X, y = data_import(path=data_path, num_of_images=200)

    processor = Ai4MarsProcessor(X, y)
    train_set, test_set, val_set = processor([0.54, 0.26, 0.20])

else:
    X = np.load('/content/dataset/X.npy')
    y = np.load('/content/dataset/y.npy')

    processor = Ai4MarsProcessor()
    train_set, test_set, val_set = processor(X, y, [0.54, 0.26, 0.20])


    # train_set = torch.load("/content/drive/MyDrive/Dataset/dataset/train.pt")
    # val_set = torch.load("/content/drive/MyDrive/Dataset/dataset/val.pt")
    # test_set = torch.load("/content/drive/MyDrive/Dataset/dataset/test.pt")



#COLPA MIAA (MY FAULT)!!!!!!!
train_set.setDevice(device)
val_set.setDevice(device)
test_set.setDevice(device)

train_set.conversion('f')
test_set.conversion('f')
val_set.conversion('f')

train_set.set_grad()
test_set.set_grad()
val_set.set_grad()

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=5, shuffle=True)
val_loader = DataLoader(val_set, batch_size=5, shuffle=True)

In [17]:
print(len(train_set))
print(len(train_loader))
print(train_loader.dataset.__getitem__(0)[0].shape)

108
2
torch.Size([3, 1024, 1024])


In [18]:
from semseg import show_models

show_models()

  No.  Model Names
-----  -------------
    1  SegFormer
    2  Lawin
    3  SFNet
    4  BiSeNetv1
    5  DDRNet
    6  FCHarDNet
    7  BiSeNetv2


In [19]:
allow_download = False

if allow_download:
    import gdown
    from pathlib import Path

    ckpt = Path('./checkpoints/pretrained/segformer')
    ckpt.mkdir(exist_ok=True, parents=True)

    url = 'https://drive.google.com/uc?id=1-OmW3xRD3WAbJTzktPC-VMOF5WMsN8XT'
    output = './checkpoints/pretrained/segformer/segformer.b3.ade.pth'

    gdown.download(url, output, quiet=False)

In [20]:
#  Import segformer

from semseg.models import *

model = eval('SegFormer')(
    backbone='MiT-B3',
    num_classes=150
)

try:
    model.load_state_dict(torch.load('checkpoints/pretrained/segformer/segformer.b3.ade.pth', map_location='cpu'))
except:
    print("Download a pretrained model's weights from the result table.")
model.eval()

model.to(device)

print('Loaded Model')

Download a pretrained model's weights from the result table.
Loaded Model


In [ ]:
#!unzip /content/drive/MyDrive/Dataset/dataset.zip -d /content/drive/MyDrive/Dataset/

In [21]:
#control to delete
print(len(train_set))

image, label = train_loader.dataset.__getitem__(0)
print(image.requires_grad)

108
True


In [ ]:
#START TRAIN

loss_fn =  torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters())
training_set = train_loader
test_set =  test_loader

trainer = Ai4MarsTrainer(loss_fn, optimizer, training_set, test_set)

In [24]:

image, label = next(iter(train_loader))

print(f'image shape: {image.shape}')


prediction = model(image)
print(label.type())
label = label.type(torch.FloatTensor)
print(label.type())


print(prediction.shape)
new_prediction = torch.argmax(prediction, dim=1)
print(f"prediction BEFORE shape -> {new_prediction.shape}")

new_prediction = new_prediction[:,None, :, :]
print(f"prediction AFTER shape -> {new_prediction.shape}")
print(f"label shape -> {label.shape}")

print(new_prediction.type())
new_prediction = new_prediction.type(torch.FloatTensor)
print(label.type())

print(loss_fn(new_prediction, label))

OutOfMemoryError: ignored

In [ ]:
trainer.train_multiple_epoch(model,EPOCHS=1)


EPOCH: 1


RuntimeError: ignored